In [7]:
import prepare_circuit as pc
import pyzx as zx
import statistics
import multiprocessing
import multiprocess as mp
# code by Y.Suzuki
from __future__ import annotations
from qiskit import QuantumCircuit, assemble
import numpy as np
import sys
def _parse_qobj_dict(qasm: str) -> dict:
    """Parse qasm to qobj_dict

    Args:
        qasm (str): qasm string
    Returns:
        dict: qobj dict
    """
    qiskit_circuit = QuantumCircuit.from_qasm_str(qasm)
    qobj = assemble(qiskit_circuit)
    qobj_dict = qobj.to_dict()
    return qobj_dict
def _check_angle(value: float) -> str:
    """Check type of rotation angle
    Args:
        value (float): rotation angle
    Returns:
        str: Any of "T", "Clifford", "Pauli", "Identity"
    """
    eps = 1e-10
    value = value % (2*np.pi)
    assert(-eps < value and value < 2*np.pi+eps)
    if abs(value - np.pi/4) < eps or abs(value - 3*np.pi/4) < eps or abs(value - 5*np.pi/4) < eps or abs(value - 7*np.pi/4) < eps:
        return "T"
    elif abs(value - 2*np.pi/4) < eps or abs(value - 6*np.pi/4) < eps:
        return "Clifford"
    elif abs(value - 4*np.pi/4) < eps:
        return "Pauli"
    elif abs(value) < eps or abs(value - 8*np.pi/4) < eps:
        return "Identity"
    else:
        return "Unknown"
def _evaluate(num_qubit: int, instructions: list) -> tuple[int, int]:
    """Retrun T-gate count and depth
    Args:
        num_qubit (int): num of qubits
        instructions (list): list of Qobj instructions
    Returns:
        tuple[int, int]: pair of T-gate count and depth
    """
    depth_counter = [0 for _ in range(num_qubit)]
    tgate_count = 0
    for inst in instructions:
        if inst["name"] in ["rz", "ry", "rz"]:
            assert(len(inst["params"]) == 1)
            assert(len(inst["qubits"]) == 1)
            angle = inst["params"][0]
            target = inst["qubits"][0]
            angle_type = _check_angle(angle)
            if angle_type == "T":
                tgate_count += 1
                depth_counter[target] += 1
            elif angle_type == "Unknown":
                print(f"Unexpected rotation angle: {inst}", file=sys.stderr)
        elif inst["name"] in ["t", "tdg"]:
            assert(len(inst["qubits"]) == 1)
            target = inst["qubits"][0]
            tgate_count += 1
            depth_counter[target] += 1
        elif inst["name"] in ["h", "s", "sdg", "x", "y", "z"]:
            continue
        elif inst["name"] in ["cz", "cx"]:
            assert(len(inst["qubits"]) == 2)
            target0 = inst["qubits"][0]
            target1 = inst["qubits"][1]
            sync_count = max(depth_counter[target0], depth_counter[target1])
            depth_counter[target0] = sync_count
            depth_counter[target1] = sync_count
        else:
            print(f"Unexpected instruction: {inst}", file=sys.stderr)
    tgate_depth = max(depth_counter)
    return tgate_count, tgate_depth
def count_tgate(qasm: str) -> tuple[int, int]:
    """count tgate from qasm string
    Args:
        qasm (str): qasm string
    Returns:
        tuple[int, int]: tgate_count and tgate_depth
    """
    qobj_dict = _parse_qobj_dict(qasm)
    num_qubit = qobj_dict["config"]["n_qubits"]
    assert(len(qobj_dict["experiments"]) == 1)
    instructions = qobj_dict["experiments"][0]["instructions"]
    tgate_count, tgate_depth = _evaluate(num_qubit, instructions)
    return tgate_count, tgate_depth
# code by R.Tokami
def count_t_depth(json):
    g = zx.graph.graph_s.GraphS.from_json(json)
    zx.full_reduce(g)
    g.normalize()
    c_opt = zx.extract_circuit(g.copy())
    print(c_opt.stats())
    after = pc.count_t_depth(c_opt.to_graph().to_json())
    return after
def experiment(size, count):
    jsons = pc.uniform_layered(size, count)
    before = pc.count_t_depth(jsons[0])
    with mp.Pool(multiprocessing.cpu_count()) as pool:
        afters = pool.map(count_t_depth, jsons)
    mean = statistics.mean(afters)
    stdev = statistics.pstdev(afters)
    return {"size":size, "count": count, "before": before, "mean": mean, "stdev": stdev}
def print_result(result_dict):
    size = result_dict["size"]
    count = result_dict["count"]
    before = result_dict["before"]
    mean = result_dict["mean"]
    stdev = result_dict["stdev"]
    print("stat: size ->", 2**size, ", count ->", count)
    print("before t-depth:", before)
    print("after t-depth:", mean, "+/-", stdev)
    imp = (1 - mean/before)*100
    print("improvement: ", imp, "%")
# code of qasm
def reduce_qasm(qasm_str):
    qasm_circuit = zx.Circuit.from_qasm(qasm_str)
    g = qasm_circuit.to_graph()
    zx.draw(g) # draw circuit
    zx.full_reduce(g)
    g.normalize()
    c_opt = zx.extract_circuit(g.copy())
    # after_qasm = c_opt.to_qasm()
    # return count_tgate(after_qasm)[1]
    after = pc.count_t_depth(c_opt.to_graph().to_json())
    return after
def experiment_qasm(size, count):
    qasms = pc.uniform_layered_qasm(size, count)
    # before = statistics.mean(list(map(lambda qasm: count_tgate(qasm)[1], qasms)))
    before = statistics.mean(list(map(lambda qasm_str: pc.count_t_depth(zx.Circuit.from_qasm(qasm_str).to_graph().to_json()), qasms)))
    with mp.Pool(multiprocessing.cpu_count()) as pool:
        afters = pool.map(reduce_qasm, qasms)
    mean = statistics.mean(afters)
    stdev = statistics.pstdev(afters)
    return {"size":size, "count": count, "before": before, "mean": mean, "stdev": stdev}


In [8]:
qasm = pc.uniform_layered_qasm(4, 1)[0]
before = count_tgate(qasm)[1]
after = reduce_qasm(qasm)
print(before, after)

92 85


In [2]:
result_test = experiment_qasm(4, 10)
print_result(result_test)

stat: size -> 16 , count -> 10
before t-depth: 92
after t-depth: 85 +/- 0.0
improvement:  7.608695652173914 %


In [18]:
result_test = experiment_qasm(5, 10)
print_result(result_test)

stat: size -> 32 , count -> 10
before t-depth: 188
after t-depth: 204.1 +/- 3.6728735344413916
improvement:  -8.563829787234045 %


In [ ]:
result_test = experiment(6, 10)
print_result(result_test)

In [6]:
result_16 = experiment(4, 100)
print_result(result_16)

In [7]:
result_32 = experiment(5, 100)
print_result(result_32)

In [8]:
result_64 = experiment(6, 100)
print_result(result_64)